# Exp2：基于回归分析的大学综合得分预测
---

## 一、案例简介
大学排名是一个非常重要同时也极富挑战性与争议性的问题，一所大学的综合实力涉及科研、师资、学生等方方面面。目前全球有上百家评估机构会评估大学的综合得分进行排序，而这些机构的打分也往往并不一致。在这些评分机构中，世界大学排名中心（Center for World University Rankings，缩写CWUR）以评估教育质量、校友就业、研究成果和引用，而非依赖于调查和大学所提交的数据著称，是非常有影响力的一个。

本任务中我们将根据 CWUR 所提供的世界各地知名大学各方面的排名（师资、科研等），一方面通过数据可视化的方式观察不同大学的特点，另一方面希望构建机器学习模型（线性回归）预测一所大学的综合得分。

## 二、作业说明
使用来自 Kaggle 的[数据](https://www.kaggle.com/mylesoneill/world-university-rankings?select=cwurData.csv)，构建「线性回归」模型，根据大学各项指标的排名预测综合得分。

**基本要求：**
* 按照 8:2 随机划分训练集测试集，用 RMSE 作为评价指标，得到测试集上线性回归模型的 RMSE 值；
* 对线性回归模型的系数进行分析。

**扩展要求：**
* 对数据进行观察与可视化，展示数据特点；
* 尝试其他的回归模型，对比效果；
* 尝试将离散的地区特征融入线性回归模型，并对结果进行对比。

**注意事项：**
* 基本输入特征有 8 个：`quality_of_education`, `alumni_employment`, `quality_of_faculty`, `publications`, `influence`, `citations`, `broad_impact`, `patents`；
* 预测目标为`score`；
* 可以使用 sklearn 等第三方库，不要求自己实现线性回归；
* 需要保留所有数据集生成、模型训练测试的代码；

## 三、数据概览

假设数据文件位于当前文件夹，我们用 pandas 读入标准 csv 格式文件的函数`read_csv()`将数据转换为`DataFrame`的形式。观察前几条数据记录：

In [180]:
import pandas as pd
import numpy as np

data_df = pd.read_csv('./cwurData.csv')  # 读入 csv 文件为 pandas 的 DataFrame
data_df.head(3).T  # 观察前几列并转置方便观察

,0,1,2
world_rank,1,2,3
institution,Harvard University,Massachusetts Institute of Technology,Stanford University
region,USA,USA,USA
national_rank,1,2,3
quality_of_education,7,9,17
alumni_employment,9,17,11
quality_of_faculty,1,3,5
publications,1,12,4
influence,1,4,2
citations,1,4,2


去除其中包含 NaN 的数据，保留 2000 条有效记录。

In [181]:
data_df = data_df.dropna()  # 舍去包含 NaN 的 row
len(data_df)

2000

取出对应自变量以及因变量的列，之后就可以基于此切分训练集和测试集，并进行模型构建与分析。

In [182]:
feature_cols = ['quality_of_faculty', 'publications', 'citations', 'alumni_employment', 
                'influence', 'quality_of_education', 'broad_impact', 'patents']
X = data_df[feature_cols]
Y = data_df['score']
X

,quality_of_faculty,publications,citations,alumni_employment,influence,quality_of_education,broad_impact,patents
200,1,1,1,1,1,1,1.0,2
201,4,5,3,2,3,11,4.0,6
202,2,15,2,11,2,3,2.0,1
203,5,10,12,10,9,2,13.0,48
204,10,11,11,12,12,7,12.0,16
...,...,...,...,...,...,...,...,...
2195,218,926,812,567,845,367,969.0,816
2196,218,997,645,566,908,236,981.0,871
2197,218,830,812,549,823,367,975.0,824
2198,218,886,812,567,974,367,975.0,651


In [183]:
X.describe()

,quality_of_faculty,publications,citations,alumni_employment,influence,quality_of_education,broad_impact,patents
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000
mean,191.127500,500.415000,449.341500,385.263500,500.21900,296.001500,496.699500,470.321000
std,52.402579,288.674823,250.141228,171.874782,288.30505,106.868798,286.919755,259.625408
min,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
25%,210.000000,250.750000,234.000000,250.750000,250.75000,250.750000,250.500000,242.750000
50%,210.000000,500.500000,428.000000,478.000000,500.50000,355.000000,496.000000,481.000000
75%,218.000000,750.000000,645.000000,500.250000,750.25000,367.000000,741.000000,737.000000
max,218.000000,1000.000000,812.000000,567.000000,991.00000,367.000000,1000.000000,871.000000


## 四、模型构建

### 代码实现

In [184]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 数据集划分
(X_train, X_test, Y_train, Y_test) = train_test_split(X, Y, test_size=0.2, random_state=20250613)

# 创建训练测试流水线，保证过程一致
model_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])


print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
# 流水下训练
model_pipeline.fit(X_train, Y_train)

Y_pred = model_pipeline.predict(X_test)
model = model_pipeline.named_steps['regressor']
rmse = root_mean_squared_error(Y_test, Y_pred)
print(f"RMSE: {rmse}")
print('intercept:', model.intercept_)
print('coef:', model.coef_)



(1600, 8) (400, 8) (1600,) (400,)
RMSE: 3.3648277730929994
intercept: 47.14546875
coef: [-3.4078841   0.09387015 -0.00388037 -1.21878317  0.22126751 -0.72524395
 -0.71556517 -0.71444118]


### 系数分析



In [185]:
pd.DataFrame(model.coef_, index=feature_cols, columns=['coef']).sort_values(by='coef', ascending=False, key=lambda x: x.abs())

,coef
quality_of_faculty,-3.407884
alumni_employment,-1.218783
quality_of_education,-0.725244
broad_impact,-0.715565
patents,-0.714441
influence,0.221268
publications,0.093870
citations,-0.003880


因为上述8个特征经过标准化（z-score归一化），因此系数的大小可以一定程度上表达特征与score的相关性。
特征分析：
1. 最关键的两个特征：`quality_of_faculty`（师资质量排名）、`alumni_employment`（校友就业排名）两个特征相关性最为显著，因为是排名因此排名越高数值越小，因此呈现较强的负相关。
2. 较为相关的4个特征：`quality_of_education`（教育质量排名）、`broad_impact`（学校广泛影响力排名）、`patents`（专利排名）、`influence`（学校“软”影响力排名）。其中前三个是较为直接的感知排名，因此排名呈现负相关。最后一个特征`influence`（学校“软”影响力排名）是潜移默化的影响力排名，这个呈现了较弱的正相关性（“软”影响力的负相关性），说明人们排名标准忽视了这个特征重要性
3. `publications`、`citations` 发版和材料引用的相关性较为微弱

## 实验报告总结

最终 RMSE=3.3648277730929994，参考模型构建部分。


这次实验又加深了对实验流程的学习。总共分为以下几个阶段。
1. 了解数据背景
2. 确认实验目标
3. 概览数据
   1. 数据筛选
   2. 确认分析选取特征
   3. 了解特征的分布情况
4. 模型构建以及模型评估

### 对于数据筛选

- 了解到了数据特征选择的重要性。特征的选取甚至可以决定最终模型训练的好坏。
- 一定不要忘记根据情况去除/填补非法数据

### 模型构建

- 对于归回分析，最终模型的系数决定了特征与目标的相关性。而特征的归一化能够使得系数绝对值大小更有意义，换句话说特征数值分散没有归一化训练的模型系数无法准确得出相关性的强弱关系。
- 回归模型的构建
  - 可以通过`pipeline`模块对数据归一化和模型训练检测提供统一的流程控制
  - 对于数据的归一化参数，样本均值和方差只能取决于训练集不取决于测试集，但是测试集要用训练集的归一化参数进行归一化。这样可以避免过拟合测试集提高模型泛化能力。


## 其他回归模型尝试

In [192]:

from sklearn.neural_network import MLPRegressor

mlp = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=1000, random_state=20250613))
])
mlp.fit(X_train, Y_train)
Y_pred_mlp = mlp.predict(X_test)

rmse_mlp = root_mean_squared_error(Y_test, Y_pred_mlp)
print(f"RMSE (MLP): {rmse_mlp}")

RMSE (MLP): 1.9072487618199085


In [193]:
from sklearn.ensemble import RandomForestRegressor
rf = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=20250613))
])
rf.fit(X_train, Y_train)
Y_pred_rf = rf.predict(X_test)

rmse_rf = root_mean_squared_error(Y_test, Y_pred_rf)
print(f"RMSE (Random Forest): {rmse_rf}")


RMSE (Random Forest): 0.6863567301520362


In [194]:
from sklearn.svm import SVR

svr = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', SVR(kernel='rbf', C=100, gamma=0.1))
])
svr.fit(X_train, Y_train)
Y_pred_svr = svr.predict(X_test)

rmse_svr = root_mean_squared_error(Y_test, Y_pred_svr)
print(f"RMSE (SVR): {rmse_svr}")

RMSE (SVR): 1.5959793677706355


In [195]:
from sklearn.tree import DecisionTreeRegressor

dt = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', DecisionTreeRegressor(max_depth=10, random_state=20250613))
])
dt.fit(X_train, Y_train)
Y_pred_dt = dt.predict(X_test)

rmse_dt = root_mean_squared_error(Y_test, Y_pred_dt)
print(f"RMSE (Decision Tree): {rmse_dt}")




RMSE (Decision Tree): 1.1420944409326614


In [196]:
from sklearn.ensemble import GradientBoostingRegressor

gb = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=20250613))
])
gb.fit(X_train, Y_train)
Y_pred_gb = gb.predict(X_test)

rmse_gb = root_mean_squared_error(Y_test, Y_pred_gb)
print(f"RMSE (Gradient Boosting): {rmse_gb}")

RMSE (Gradient Boosting): 0.8125646307697291


In [197]:
from sklearn.ensemble import AdaBoostRegressor

ab = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', AdaBoostRegressor(n_estimators=100, learning_rate=0.1, random_state=20250613))
])
ab.fit(X_train, Y_train)
Y_pred_ab = ab.predict(X_test)

rmse_ab = root_mean_squared_error(Y_test, Y_pred_ab)
print(f"RMSE (AdaBoost): {rmse_ab}")

RMSE (AdaBoost): 1.4653352333303078
